In [2]:

%reload_ext autoreload
%autoreload 2
import ast
import pickle
import itertools
from collections import Counter
from tqdm import tqdm
import pandas as pd
import os

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import cv2
import swifter

import sys
sys.path.append('..')
from analysis.generate_cluster_information_file import load, extract_all_information_query, to_df_query
from common.image_processing import pixel_intensity_histogram


from sklearn.model_selection import cross_val_score
from sklearn.ensemble import IsolationForest, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, make_scorer
# import xgboost
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV

In [23]:
train_X = pd.read_csv('train_X.csv')
train_y = pd.read_csv('train_y.csv')

pd.to_numeric(train_X['cluster_num'])
pd.to_numeric(train_X['cluster_num_intensities_avg'])
pd.to_numeric(train_X['cluster_peak_intensities_avg'])
pd.to_numeric(train_X['cluster_x_avg'])
pd.to_numeric(train_X['cluster_y_avg'])

na_indices = train_X[train_X.isna().any(axis=1)].index.values
train_X.drop(na_indices, inplace=True)
train_y.drop(na_indices, inplace=True)

print(train_X.shape)
train_X.head(10)

(9599, 5)


,cluster_num,cluster_num_intensities_avg,cluster_peak_intensities_avg,cluster_x_avg,cluster_y_avg
0,50.0,6.680000,14.920000,513.460000,475.320000
1,104.0,5.692308,11.375000,536.201923,473.798077
2,71.0,5.605634,11.563380,464.971831,513.746479
3,49.0,8.428571,22.612245,417.285714,564.571429
4,169.0,3.284024,6.621302,462.224852,509.230769
5,51.0,10.450980,26.921569,590.921569,492.352941
6,2216.0,3.131318,196.498195,433.569495,468.828520
7,66.0,6.409091,15.045455,450.257576,427.469697
8,73732.0,2.120314,17.661884,498.615499,497.128208
9,35.0,6.257143,14.171429,540.771429,514.942857


In [35]:
def train_with_features(features):
    assert len(features) > 0
    assert type(features[0]) == str
    
    X = train_X[features]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    y_scaled = scaler.fit_transform(train_y)
    
    randomforest = RandomForestRegressor(max_depth=6, random_state=0)

    score_mi = cross_val_score(randomforest, X_scaled, np.ravel(y_scaled), cv=5, scoring=make_scorer(mean_absolute_error))
    print("Mean: {}, Std Dev: {}".format(score_mi.mean(), np.std(score_mi)))

In [36]:
train_with_features(['cluster_num'])

Mean: 0.6488447391468939, Std Dev: 0.010385121122722645


In [37]:
train_with_features(['cluster_num_intensities_avg'])

Mean: 0.6965943964912368, Std Dev: 0.00942424878893548


In [38]:
train_with_features(['cluster_peak_intensities_avg'])

Mean: 0.6354252098067206, Std Dev: 0.013436356146284424
